In [ ]:
import pandas as pd
from tqdm import tqdm
import string
import json
import copy
import re

import os

In [ ]:
data = pd.read_csv('data/work_arrangements_development_set.csv')

In [ ]:
data

,id,job_ad,y_true
0,79484040,Job title: CEO\nAbstract: Exciting opportunity...,Remote
1,80331384,Job title: Home-Based Online ESL Teacher (Onli...,Remote
2,79721069,"Job title: Safeguarding, De La Salle\nAbstract...",Hybrid
3,80190376,Job title: Delivery Driver\nAbstract: Pickup t...,OnSite
4,80082230,Job title: Store Supervisor\nAbstract: We are ...,OnSite
...,...,...,...
94,80151196,Job title: Senior Pipeline Technical Director\...,Hybrid
95,79652545,Job title: Customer Support Administrator\nAbs...,OnSite
96,79718230,Job title: Remote Writing Evaluator for AI (As...,Remote
97,80226188,Job title: People & Culture Advisor\nAbstract:...,Hybrid


In [ ]:
train_df = pd.read_csv("data/work_arrangements_development_set.csv")
test_df = pd.read_csv("work_arrangements_test_set.csv")
train_df.iloc[48]

,48
id,80058389
job_ad,Job title: Tax Agent Training and Mentor Manag...
y_true,Hybrid


# Loading Qwen

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
def call_model(messages):
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return response

## rbic functions

In [ ]:
#== step 1: role explanation
def step_1():
  messages = [
        {"role": "system", "content": "You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite."},
        {"role": "user", "content": "Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?"}
  ]

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  return response[0], messages

In [ ]:
#== few-shot examples
def fewshot(messages):
    few_shot_indices = [20,23,36,48]

    for i in few_shot_indices:
        desc = {
          "job_ad": train_df.iloc[i].job_ad
        }
        desc_str = str(desc)

        # add the description
        messages.append({
          "role": "user",
          "content": desc_str
        })

        label = train_df.iloc[i].y_true
        label_str = str(label)

        # add the output
        messages.append({
            "role": "assistant",
            "content": label_str
        })

    return messages

In [ ]:
fewshot([])

[{'role': 'user',
  'content': "{'job_ad': 'Job title: Senior Healthcare Recruiter/Project Manager Assistant (US Shift, WFH)\\nAbstract: Hiring for a US Healthcare Recruiter with Project Management experience! Apply now and be hired right away!\\nEmployer: Go Virtual\\nLocation: -\\nHighlights: \\nContents: Job Description:\\n Recruitment Strategy and Planning, Candidate Sourcing and Outreach, Screening and Interviewing, Offer Management and Onboarding, Employer Branding and Relationship Management, Metrics and Reporting, Project Coordination, Communication, Budget and Financial Monitoring, Quality Assurance.\\n Qualifications:\\n Bachelorâ€™s degree in Human Resources, Business Administration, or a related field, A minimum of 5-10 years of experience in healthcare (clinical & non-clinical), administrative, and professional (subject matter experts, HR professionals) support services recruitment, Proven track record of successfully recruiting healthcare professionals in a fast-paced env

In [ ]:
#== step 2: setting sub-task --> ask for seniority patterns
def step_2(messages):
    messages.append({
        "role": "user",
        "content": "As a work arrangements classifier, what are some common phrases or patterns that indicate a work arrangements in a job description?"
    })

    response = call_model(messages)
    messages.append({"role": "assistant", "content": response[0]})

    return response[0], messages

In [ ]:
#== step 3: presence of seniority (skipped)
def step_3(messages_static, desc_str):
    messages = copy.deepcopy(messages_static)

    messages.append({
        "role": "user",
        "content": f"{desc_str} does this job description include any work-arrangements related information? Just respond with 'Yes' or 'No'."
    })

    response = call_model(messages)
    messages.append({"role": "assistant", "content": response[0]})

    p3_content = response[0]

    # clean and check the response
    response_p3 = p3_content.translate(str.maketrans('', '', string.punctuation))
    response_p3 = response_p3.strip().lower()[:3]
    #print(f"step3 messages: {len(messages)}")
    return response_p3, messages

In [ ]:
#== step 4: iterative coaching/finding clues to prevent hallucination
def step_4(response_p3, messages):
    messages.append({
            "role": "user",
            "content": "Extract the work arrangements related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
        })

    response = call_model(messages)
    messages.append({"role": "assistant", "content": response[0]})
    #print(f"step4 messages: {len(messages)}")

    return response[0], messages

In [ ]:
#== step 5: use the clue to generate the final output
def step_5(messages):
    messages.append({
        "role": "user",
        "content": (
            "Based on the extracted work-arrangement clue and the text verbatim, return a structured seniority label in the format {\"work_arrangements_label\": \"work arrangements classification\"}. Return only the json object with no extra details."

        )
    })

    response = call_model(messages)
    answer_str = response[0]

    print(messages)

    return answer_str

In [ ]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
    response, messages = step_1()
    if verbose: print(f"RB step 1: {response}\n")

    response, messages = step_2(messages)
    if verbose: print(f"RB step 2: {response}\n")

    if add_fewshot:
        messages = fewshot(messages)
        if verbose: print(f"Fewshot examples added\n")
    return messages

In [ ]:
def RBIC(messages, desc_str, verbose=False):
    response_p3, messages_local = step_3(messages, desc_str)
    if verbose: print(f"IC step 1: {response_p3}\n")

    response, messages_local = step_4(response_p3, messages_local)
    if verbose: print(f"IC step 2: {response}\n")

    answer_str = step_5(messages_local)
    match = re.search(r'\{.*\}', answer_str)
    if match:
        json_str = match.group(0)
        data = json.loads(json_str)
        try:
            label = data["work_arrangements_label"]
        except:
            label = "ERROR" + str(data)
    else:
        # Handle the case where no JSON is found, e.g., by setting a default label
        label = "Unknown"  # or any other suitable default



    if verbose: print(f"IC step 3 (Final): {label}\n")
    if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

    return str(label)

# Qualitative Testing RBIC + fewshot

In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: Work arrangement refers to the way in which employees are scheduled for their roles within a company or organization. It encompasses factors such as hours worked, days off, vacation time, and any other arrangements that affect how employees interact with each other, manage their personal lives, and participate in organizational activities.

Labeling work arrangements typically includes terms like "on-site," "remote," "hybrid," and others based on specific characteristics such as physical location, communication methods, or working schedule. These labels help organizations understand and adapt to different types of work environments effectively.



In [ ]:
ind = 2

#[2081,1,5,12,3]
''' desc = {
          "job_ad": train_df.iloc[i].job_ad
        }'''


desc = {
    "job_ad": test_df.iloc[ind].job_ad
  }
desc_str = str(desc)

label = test_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 13
IC step 1: yes

IC step 2: ```json
{"Clue": "remote", "Explanation": "The job describes the position as a "Senior Healthcare Recruiter/Project Manager Assistant (US Shift, WFH)""}
```

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrang

In [ ]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df

,y_pred


In [ ]:
test_pred_df2 = pd.DataFrame(columns=["y_pred"])
test_pred_df2

,y_pred


In [ ]:
range(len(test_df))

range(0, 99)

## Quantitative testing

In [ ]:
for i in tqdm(range(len(test_df))):

    desc = {
        "job_ad": train_df.iloc[i].job_ad
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred

  1%|          | 1/99 [00:05<08:39,  5.30s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrangements typically includes terms like "Onsite," "Remote," "Full-Time," "Part-Time," "Temporary," "Fixed-term," "Contractual," "Flexible," and "Hourly." These labels help in identifying the nature of work arrangem

  2%|▏         | 2/99 [00:11<09:32,  5.90s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrangements typically includes terms like "Onsite," "Remote," "Full-Time," "Part-Time," "Temporary," "Fixed-term," "Contractual," "Flexible," and "Hourly." These labels help in identifying the nature of work arrangem

  3%|▎         | 3/99 [00:16<08:58,  5.61s/it]

...

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrangements typically includes terms like "Onsite," "Remote," "Full-Time," "Part-Time," "Temporary," "Fixed-term," "Contractual," "Flexible," and "Hourly." These labels help in identifying the nature of work arrangem

 99%|█████████▉| 98/99 [10:11<00:06,  6.07s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrangements typically includes terms like "Onsite," "Remote," "Full-Time," "Part-Time," "Temporary," "Fixed-term," "Contractual," "Flexible," and "Hourly." These labels help in identifying the nature of work arrangem

100%|██████████| 99/99 [10:16<00:00,  6.22s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangements refer to the methods by which employees are assigned to different jobs or departments within an organization. They include factors such as location (on-site, remote), time frame (full-time, part-time), and type of employment (permanent, temporary). Work arrangements are crucial for ensuring that employees have access to opportunities based on their needs and preferences.\n\nLabeling work arrangements typically includes terms like "Onsite," "Remote," "Full-Time," "Part-Time," "Temporary," "Fixed-term," "Contractual," "Flexible," and "Hourly." These labels help in identifying the nature of work arrangem

In [ ]:
test_pred_df

,y_pred
0,OnSite
1,OnSite
2,Onsite
3,on-site
4,Seniority Classification
...,...
109,Onsite
110,hybrid
111,OnSite
112,Onsite


In [ ]:
test_pred_df.to_csv('workarrangements_qwen_preds_few_shot.csv', index=False)

# RBIC + No few shot


In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=False)

RB step 1: Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.

Work arrangements can be classified into three main types:

1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.
   
2. **Hybrid**: Similar to remote work, but also includes some elements of traditional office work. Employers may offer some flexibility by allowing employees to work remotely, while still maintaining certain aspects of traditional office culture.
   
3. **Onsite**: This arrangement involves working in the same physical space as the primary workplace, typically in a central location. Employees are physically present at the company's premises, although the

In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_ad": train_df.iloc[i].job_ad
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred

# export the dataframe to a new csv file
test_pred_df.to_csv('workarrangements_qwen_preds_no_few_shot.csv', index=False)

  1%|          | 1/99 [00:02<04:37,  2.83s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.\n\nWork arrangements can be classified into three main types:\n\n1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.\n   \n2. **Hybrid**: Similar to remote work, but also includes some elements of traditional off

  2%|▏         | 2/99 [00:14<13:04,  8.08s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.\n\nWork arrangements can be classified into three main types:\n\n1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.\n   \n2. **Hybrid**: Similar to remote work, but also includes some elements of traditional off

  3%|▎         | 3/99 [00:16<08:46,  5.49s/it]

...

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.\n\nWork arrangements can be classified into three main types:\n\n1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.\n   \n2. **Hybrid**: Similar to remote work, but also includes some elements of traditional off

 99%|█████████▉| 98/99 [05:40<00:02,  2.72s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.\n\nWork arrangements can be classified into three main types:\n\n1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.\n   \n2. **Hybrid**: Similar to remote work, but also includes some elements of traditional off

100%|██████████| 99/99 [05:42<00:00,  3.46s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Certainly! Work arrangement refers to the physical location where employees perform their tasks. It encompasses factors such as the type of workspace (e.g., office, home), the nature of the work (e.g., remote, in-person), and the level of communication with colleagues.\n\nWork arrangements can be classified into three main types:\n\n1. **Remote**: This involves working from locations that do not match the primary workplace. Employees may work from different parts of the country or even internationally, often at home offices.\n   \n2. **Hybrid**: Similar to remote work, but also includes some elements of traditional off

# Prompting

In [ ]:
test_pred_df3 = pd.DataFrame(columns=["y_pred"])

In [ ]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
messages_static = [
    {"role": "system",  "content": "You are an expert job ad annotator. Your task is to classify a job ad into one of the work arrangements label - (OnSite, Hybrid, Remote) from job descriptions."},
]

In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {"job_ad": test_df.iloc[i].job_ad}
    desc_str = str(desc)

    # rebuild messages for this row
    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract work-arrangements information from the job ad. "
            "Return a structured work-arrangements label in the format "
            "{\"work_arrangements_label\": \"work arrangements classification\"}. "
            "Return only the JSON object with no extra details."
        )
    })

    response = call_model(messages)
    answer_str = response[0]

    # parse out the JSON
    try:
        match = re.search(r'\{.*\}', answer_str)
        json_str = match.group(0)
        data = json.loads(json_str)
        label = data["work_arrangements_label"]
    except (AttributeError, json.JSONDecodeError):
        print(f"Failed to parse JSON on row {i}: {answer_str}")
        label = "ERROR " + answer_str

    # append to the next empty row
    test_pred_df3.loc[len(test_pred_df3)] = label

# finally, save your results
test_pred_df3.to_csv(
    'work_arrangements_labelled_test_set_gemini_rbic_preds_simple_prompt.csv',
    index=False
)

100%|██████████| 99/99 [01:18<00:00,  1.26it/s]


In [ ]:
test_df

,id,job_ad,y_true
0,79746043,Job title: Restaurant Kitchen Hand\nAbstract: ...,OnSite
1,79796468,Job title: Chef\nAbstract: The Carlin Group Pt...,OnSite
2,80346773,"Job title: Accounting Associate (AU, NZ, US Ex...",Remote
3,80015193,Job title: Occupational Therapist\nAbstract: P...,Hybrid
4,80292327,Job title: Social Media Manager Paid Ads\nAbst...,Remote
...,...,...,...
94,79930630,Job title: Dietitian - Aged Care - Hervey Bay/...,OnSite
95,79847353,Job title: Customer Support Consultant - Retai...,Hybrid
96,80052768,Job title: Aluminium windows & doors - saw ope...,OnSite
97,80088290,Job title: Landscape Maintenance (Full Time) L...,OnSite


In [ ]:
test_pred_df3

,y_pred
0,Hybrid
1,OnSite
2,OnSite
3,OnSite
4,Hybrid
...,...
94,OnSite
95,Work from Home
96,OnSite
97,OnSite


# Work Arrangements Metrics

# **Post Processing**

# Prompting post processing

In [ ]:
test_df["y_true"].unique()

array(['OnSite', 'Remote', 'Hybrid'], dtype=object)

In [ ]:
prompt = pd.read_csv("work_arrangements_labelled_test_set_gemini_rbic_preds_simple_prompt.csv")
prompt["y_pred"].unique()

array(['Hybrid', 'OnSite', 'Work from Home', 'Remote', 'on-site',
       'Work-from-home', 'Part-Time', 'Work-from-Home',
       'Work arrangement classification'], dtype=object)

In [ ]:
# Make a set of class labels
prompt["y_pred"] = prompt["y_pred"].replace("on-site", "OnSite")
prompt["y_pred"] = prompt["y_pred"].replace("Work from Home", "Remote")
prompt["y_pred"] = prompt["y_pred"].replace("Work-from-home", "Remote")
prompt["y_pred"] = prompt["y_pred"].replace("Work-from-Home", "Remote")
prompt["y_pred"].unique()

array(['Hybrid', 'OnSite', 'Remote', 'Part-Time',
       'Work arrangement classification'], dtype=object)

# RBIC post processing

In [ ]:
RBIC =  pd.read_csv("workarrangements_qwen_preds_no_few_shot.csv")
RBIC["y_pred"].unique()

array(['on-site', 'Future-focused', 'work arrangements classification',
       'flexible hours', 'Hybrid', 'Full remote', 'Remote', 'Seniority',
       'flexible working hours', 'full-time', 'Contracting', 'Onsite',
       'remote', 'Full-time', 'on-the-job-training', 'Flexible'],
      dtype=object)

In [ ]:
RBIC["y_pred"] = RBIC["y_pred"].replace("on-site", "OnSite")
RBIC["y_pred"] = RBIC["y_pred"].replace("Onsite", "OnSite")
RBIC["y_pred"] = RBIC["y_pred"].replace("remote", "Remote")
RBIC["y_pred"].unique()

array(['OnSite', 'Future-focused', 'work arrangements classification',
       'flexible hours', 'Hybrid', 'Full remote', 'Remote', 'Seniority',
       'flexible working hours', 'full-time', 'Contracting', 'Full-time',
       'on-the-job-training', 'Flexible'], dtype=object)

# RBIC + few processing

In [ ]:
RBIC_few_shot = pd.read_csv("workarrangements_qwen_preds_few_shot.csv")
RBIC_few_shot["y_pred"].unique()

array(['OnSite', 'Onsite', 'on-site', 'Seniority Classification',
       'Seniority', 'Hybrid', 'On-site', 'hybrid', 'Onsite, Remote',
       'remote', 'On-Site', 'Senior', 'onsite'], dtype=object)

In [ ]:
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("on-site", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("Onsite", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("On-site", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("On-Site", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("onsite", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("Onsite", "OnSite")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("hybrid", "Hybrid")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("hybrid", "Hybrid")
RBIC_few_shot["y_pred"] = RBIC_few_shot["y_pred"].replace("remote", "Remote")

RBIC_few_shot["y_pred"].unique()

array(['OnSite', 'Seniority Classification', 'Seniority', 'Hybrid',
       'Onsite, Remote', 'Remote', 'Senior'], dtype=object)

# **Evaluation**

In [ ]:
from sklearn.metrics import classification_report, f1_score
def evaluate(preds, true):

  # 2. Extract the columns
  y_pred = preds['y_pred']
  y_true = true['y_true']

  # 3. Define your label set
  labels = ['OnSite', 'Hybrid', 'Remote']

  # 4. Print the detailed classification report
  print(classification_report(
      y_true,
      y_pred,
      labels=labels,
      target_names=labels,
      digits=3
  ))

  # 5. Compute and print key F₁ metrics
  print("Macro F₁:   ", f1_score(y_true, y_pred, average='macro', labels=labels))
  print("Weighted F₁:", f1_score(y_true, y_pred, average='weighted', labels=labels))
  print("Micro F₁:   ", f1_score(y_true, y_pred, average='micro', labels=labels))

In [ ]:
def accuracy(test_df, prompt):
  #Accuracy
  total    = len(test_df)
  excat_match = (test_df['y_true'] == prompt['y_pred']).sum()
  print("total", total)
  print(f"Exact matches:        {excat_match}  ({excat_match/total:.1%})")

## **Prompting**

In [ ]:
evaluate(prompt, test_df)

              precision    recall  f1-score   support

      OnSite      0.577     0.652     0.612        46
      Hybrid      0.395     0.556     0.462        27
      Remote      0.857     0.231     0.364        26

   micro avg      0.526     0.515     0.520        99
   macro avg      0.610     0.479     0.479        99
weighted avg      0.601     0.515     0.506        99

Macro F₁:    0.4791399077113363
Weighted F₁: 0.5058520123455189
Micro F₁:    0.5204081632653061


In [ ]:
accuracy(test_df, prompt)

total 99
Exact matches:        51  (51.5%)


## **RBIC**

In [ ]:
evaluate(RBIC, test_df)

              precision    recall  f1-score   support

      OnSite      0.429     0.326     0.370        46
      Hybrid      0.300     0.111     0.162        27
      Remote      0.250     0.038     0.067        26

   micro avg      0.388     0.192     0.257        99
   macro avg      0.326     0.159     0.200        99
weighted avg      0.347     0.192     0.234        99

Macro F₁:    0.19973306639973307
Weighted F₁: 0.23382574493685604
Micro F₁:    0.25675675675675674


In [ ]:
accuracy(test_df, RBIC)

total 99
Exact matches:        19  (19.2%)


# **RBIC + no few shot**

In [ ]:
evaluate(RBIC_few_shot[:99], test_df)

              precision    recall  f1-score   support

      OnSite      0.488     0.870     0.625        46
      Hybrid      0.500     0.074     0.129        27
      Remote      0.000     0.000     0.000        26

   micro avg      0.483     0.424     0.452        99
   macro avg      0.329     0.315     0.251        99
weighted avg      0.363     0.424     0.326        99

Macro F₁:    0.2513440860215054
Weighted F₁: 0.32559465623981754
Micro F₁:    0.45161290322580644


In [ ]:
accuracy(test_df, RBIC_few_shot[:99])

total 99
Exact matches:        42  (42.4%)
